In [ ]:
# Log fordeling?
# Samme person må ikke være i både train og test. Hvad med de andre ting? Hvorfor?

In [8]:
import sys
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd

In [11]:
data = pd.read_csv("HR_data.csv")
data = data.astype({"Frustrated": str})
data

y = data["Frustrated"].values
x = data[["HR_Mean", "HR_Median", "HR_std", "HR_Min", "HR_Max", "HR_AUC"]].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train = torch.tensor(x_train.astype(np.float32))
x_test = torch.tensor(x_test.astype(np.float32))
y_train = torch.tensor(y_train.astype(np.float32))
y_test = torch.tensor(y_test.astype(np.float32))

def one_hot_encode(input_tensor, num_classes=None):
    if num_classes is None:
        num_classes = len(torch.unique(input_tensor))
    one_hot_encoded = torch.nn.functional.one_hot(input_tensor.long(), num_classes)
    return one_hot_encoded

y_train = one_hot_encode(y_train, num_classes=11)
y_test = one_hot_encode(y_test, num_classes=11)

y_train = y_train.type(torch.float)
y_test = y_test.type(torch.float)

In [28]:

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(6, 11)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        return self.activation(x)

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

def eval_model(model, x_test, y_test):
    model.eval()
    raw_predictions = model(x_test)
    # print(raw_predictions[:10])
    predictions = np.argmax(raw_predictions.detach().numpy(), axis=1)
    y_test = np.argmax(y_test, axis=1)
    return accuracy_score(predictions, y_test)

# Train the model
for epoch in range(100):
    # if epoch % 10 == 0: print(eval_model(model, x_test, y_test))
    for i in range(len(y_train)):
        model.train()
        optimizer.zero_grad()
        outputs = model(x_train[i])
        loss = criterion(outputs, y_train[i])
        loss.backward()
        optimizer.step()

In [30]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 613.8560180664062
199 411.795654296875
299 277.3367919921875
399 187.82652282714844
499 128.21377563476562
599 88.49449157714844
699 62.017906188964844
799 44.35993957519531
899 32.57731628417969
999 24.71072006225586
1099 19.45570182800293
1199 15.943110466003418
1299 13.593765258789062
1399 12.021403312683105
1499 10.96829605102539
1599 10.262483596801758
1699 9.78908920288086
1799 9.47130298614502
1899 9.257822036743164
1999 9.114290237426758
Result: y = 0.00990060530602932 + 0.8426662683486938 x + -0.0017080189427360892 x^2 + -0.09132839739322662 x^3
